In [78]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

from nltk.stem.porter import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [79]:
dataf=pd.read_csv('cleaned_data.csv')

In [80]:
df1 = dataf.copy()
df1=df1[['uid','synopsis']]

In [81]:
to_remove=df1[df1['synopsis']=='No synopsis information has been added to this title. Help improve our database by adding a synopsis'].index.to_list()
df1=df1.drop(df1.index[[to_remove]])

In [82]:
to_remove=df1[df1['synopsis']=='No synopsis information has been added to this title. Help improve our database by adding a synopsis'].index.to_list()
print(to_remove)

[]


In [83]:
stop = set(stopwords.words('english'))
def lemmatize(text):
    try: 
        sentence = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
        text = sentence.sub(" ", text) 
        
        word_tokens = [word_tokenize(i) for i in sent_tokenize(text)]
        
        tokens = []
        for j in word_tokens:
            tokens += j
            
        tokens = list(filter(lambda t: t.lower() not in stop, tokens))
        lowcase = [w for w in tokens if re.search('[a-zA-Z]', w)]
        lowcase = [w.lower() for w in lowcase if len(w)>=3]
        
        lemmatizer = WordNetLemmatizer()
        lemmatized = [lemmatizer.lemmatize(word) for word in lowcase if len(word)>1]
        return lemmatized[:-3]
            
    except TypeError:
        print(e)
        pass


In [84]:
df1['lemmat']=df1['synopsis'].map(lemmatize)

In [85]:
df1.reset_index(drop=True, inplace=True)

In [86]:
df1

,uid,synopsis,lemmat
0,28891,Following their participation at the Inter-Hig...,"[following, participation, inter, high, karasu..."
1,23273,Music accompanies the path of the human metron...,"[music, accompanies, path, human, metronome, p..."
2,34599,The Abyss—a gaping chasm stretching down into ...,"[abyss—a, gaping, chasm, stretching, depth, ea..."
3,5114,"""In order for something to be obtained, someth...","[order, something, obtained, something, equal,..."
4,31758,After helping revive the legendary vampire Kis...,"[helping, revive, legendary, vampire, kiss, sh..."
...,...,...,...
15609,10075,All-new animation offered throughout UNIQLO cl...,"[new, animation, offered, throughout, uniqlo, ..."
15610,35828,High school student Sora Kashiwagi is accustom...,"[high, school, student, sora, kashiwagi, accus..."
15611,10378,"After regaining her squid-like abilities, Ika ...","[regaining, squid, like, ability, ika, musume,..."
15612,33082,"For years, the Niflheim Empire and the kingdom...","[year, niflheim, empire, kingdom, lucis, war, ..."


In [87]:
df1.to_csv('synopsis.csv')

In [88]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_transformed = count.fit_transform(df1['synopsis'])

In [89]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similar = cosine_similarity(count_transformed, count_transformed)

In [90]:
df1 = df1.reset_index()
indices = pd.Series(df1.index, index=df1['uid'])

In [107]:
def get_recommendations(uid, cosine_sim):

    sim_scores = list(enumerate(cosine_sim[indices[uid]]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return dataf['title'].iloc[movie_indices]

In [112]:
\get_recommendations(28891, cosine_sim=cosine_similar).values[0].split(' ')

True

In [116]:
dataf['title'].iloc[[j for j in movie_indices if 'Haikyuu!!:' not in dataf['title'].iloc[j].split(' ')]]

735                                              Haikyuu!!
1673                            Hataraku Saibou 2nd Season
9330                                            Gan to Gon
3229                             Hei Mao Jing Zhang (1992)
11489                                             Paradise
447                                         Diamond no Ace
648                            Kuroko no Basket 2nd Season
10413                                              Berserk
13358    Tottoko Hamtarou: Best Hit Ham-chan'zu Music Clip
10373                                        Qualidea Code
557                                       Kuroko no Basket
14106                                           Happiness!
5884                     Diamond no Ace: Second Season OVA
626                          Diamond no Ace: Second Season
4931                                          Armitage III
1049     Girls & Panzer: Kore ga Hontou no Anzio-sen Desu!
4927                     Uchuu Senkan Yamato: Fukkatsu-h

In [106]:
indices = pd.Series(dataf.index, index=dataf['uid'])
dataf['title'].iloc[indices[28891]].split(' ')[0].replace(':','') not in 

'Haikyuu!!'

In [ ]:
import pickle

In [120]:
object_ = pd.read_pickle(r'genre.pkl')
gen_sim = cosine_similarity(object_)

In [123]:
location = dataf.loc[dataf['uid']==28891].index[0]
recommended_lst = list(enumerate(gen_sim[location]))

In [125]:
recommended_lst = sorted(recommended_lst, key = lambda x:x[1] ,reverse=True)
recommended_lst = recommended_lst[1:50]

In [127]:
df_full=dataf.copy()

In [131]:
movie_i = [j[0] for j in recommended_lst]
final_list = [i for i in movie_i if 'Season' not in df_full['title'].iloc[i]]
indices = pd.Series(df_full.index, index=df_full['uid'])
final_list=[j for j in final_list if df_full['title'].iloc[indices[28891]].split(' ')[0].replace(':','') not in df_full['title'].iloc[j].split(' ')]
recommended_series = df_full['title'].iloc[final_list]

In [132]:
recommended_series

10       Haikyuu!!: Karasuno Koukou vs. Shiratorizawa G...
705                                              Slam Dunk
1338                                         Batsu & Terry
1390                                     Rokudenashi Blues
1675                                 Haikyuu!!: To the Top
1676                               Haikyuu!!: Riku vs. Kuu
5798                                         Ahiru no Sora
539                                     Ballroom e Youkoso
659                                             Cross Game
13687                               Rokudenashi Blues 1993
176                              Kuroko no Basket: Tip Off
323        Tennis no Ouji-sama: Zenkoku Taikai-hen - Final
418                           Yowamushi Pedal: Grande Road
431                       Hajime no Ippo: Boxer no Kobushi
447                                         Diamond no Ace
502                                 Diamond no Ace: Act II
516                                         Major: Messa